In [14]:
# Our common data handling package
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import urlopen
import gc
import psutil

In [15]:
#remove any of the titles
episode=pd.read_csv('../data/episode_info.csv')
listofepisodetitles=[str(s).lower().replace(' ','') for s in episode.title]

In [16]:
#For the main characters
Monica=[]
Ross=[]
Phoebe=[]
Joey=[]
Rachel=[]
Chandler=[]
All=[]
scenes=[]
minors=[]
all_char=[['monica', True],['ross', True],['phoebe', True],['joey', True],['rachel', True],['chandler', True]]



In [17]:
# for all the seasons in the set [from_season, to_season]

from_season=1
to_season=10


#almost the number of scripts per episode
mumberofepisodes=[24, 24,25, 23, 23, 25, 23, 23, 24, 18]

#A table where our information will go for now
allwords=[]
strings="https://brookeogrodnik.github.io/scripts/{}.html"

for seas in range(from_season,to_season+1):
    for epi in range(1, mumberofepisodes[seas-1]+1):
        search=True
        val=100*seas+epi
        
        #not dealing with 1 script for 2 epsiodes here (and other places also, but only these changed the name of the file)
        if val not in [212, 213, 615, 616, 624, 625, 923, 924, 1017, 1018]:
        
            if val<1000:
                val='0'+str(val)
            else:
                val=str(val)
                
        #we already read these ones in 
        elif val in  [213, 616, 625,  924, 1018]:
            search=False
            
        #special names 
        elif val==212:
            val='0212-0213'
        elif val==615:
            val='0615-0616'    
        elif val==624:
            val='0624'
        elif val==923:
            val='0923-0924'
        elif val==1017:
            val='1017-1018'    
        
        if search:
            html = urlopen(strings.format(val))
            soup = BeautifulSoup(html,"html.parser")
            theirHTMLisWeird=soup.find_all(text=True)
            
            allwords.append([[seas, epi], theirHTMLisWeird])
gc.collect()



0

In [18]:
#gets rid of a bunch of stuff we really don't want in the script
def allowed(line_of_script):
    l=str(line_of_script).lower().replace(" ","")
    if l=='' or l=='.': return False
    if l.find('commericalbreak')>-1: return False
    if l=='openingcredits': return False
    if l=='closingcredits': return False
    if l=='\n': return False
    if l.find('<!--')>-1: return False
    
    return True

In [19]:
#more absolute junk in the begining 
def allowed_after_initial_clean(line_of_script):
    l=str(line_of_script).lower().replace(" ","")
    if l.find('note:')==0: return False
    if l.find('writtenby')>-1: return False
    if l.find('transcribedby')>-1: return False
    if l.find('transcriptby')>-1: return False
    if l.find('directedby')>-1: return False
    if l.find('transcribingby')>-1: return False
    if l.find('withminoradjustmentsby')>-1: return False
    if l.find('withminotadjustmentsby')>-1: return False
    if l.find('withhelpfrom:')>-1: return False
    if l.find('guineapig')>-1: return False
    if l.find('trascribedby')>-1: return False
    if l.find('@')>-1: return False
    if l.find('storyby:')>-1: return False
    if l.find('story:')==0: return False
    if l.find('aired:')==0: return False
    if l.find('mindymattinglyphillips')>-1: return False
    if l.find('dansilverstein')>-1: return False
    if l.find('joshhodge')>-1: return False
    if l.find('alexajunge')>-1: return False
    if l.find('iraungerleider')>-1: return False
    if l.find('brownmandell')>-1: return False
    if l.find('joshuahodge')>-1: return False
    if l.find('adamchase')>-1: return False
    if l.find('???')==0: return False
    if l.find('teleplayby')>-1: return False
    if l.find('ericbaasen')>-1: return False
    if l.find('html')>-1: return False
    if l.find('aliciaskyvarinaitis')>-1: return False
    if l.find('markj.kunerth')>-1: return False
    if l.find('ruthcurran')>-1: return False
    if l in listofepisodetitles: return False
    if l.find('-theonewith')>-1: return False
    if l.find('-theonein')>-1: return False
    if l.find('-thelastone')>-1: return False
    if l.find('-theoneafter')>-1: return False
    if l.find('-theonewhere')>-1: return False
    if l.find('914theonewith')==0: return False
    if l.find('theonewith')==0: return False
    if l.find('heonewith')==0: return False
    if l.find('theoneafter')==0: return False
    if l.find('theonethemorningafter')==0: return False
    if l.find('theonewhere')==0: return False
    if l.find('theoneinvegas')==0: return False
    if l.find('theonethelastnight')==0: return False
    if l.find('friends-')==0: return False
    if l.find('teleplay')==0: return False
    if l.find('transcriber’snote')>-1: return False
    if l.find('dutchphrasesby')>-1: return False
    if l.find('url=')>-1: return False
    if l.find('creationdate:')>-1: return False
    if l.find('generatedby')==0: return False
    if l.find('produced')==0: return False
    if l.find('untitleddocument')==0: return False
    if l.find('russiantoroman')==0: return False
    if l.find('templatebegineditable')==0: return False
    if l.find('templateendeditable')==0: return False
    if l.find('finalcheckby')==0: return False
    if l.find('bluetext')>-1: return False
    if l==',': return False
    if line_of_script=='Friends': return False
    if l.find("transcriber's")>-1: return False
    
    
    return True

In [20]:
templist=[0]*len(allwords)

#Trying to fix most of the inconsitencies
for pos in range(0,len(allwords)):

    
    episode_info=allwords[pos][0]
    
    #get rid of blank lines and the \n, change all parenthesis and other such 
    #groupings to backets then strip it so we have no white space on the outsides
    script=[str(line).replace('\n',' ').replace('\xa0',' ').replace('<','[').replace('(','[').replace('>',']').replace(')',']').strip() for line in allwords[pos][1] if allowed(line)]
    
    
    #one last runthrough to get rid of empty lines
    script=[line for line in script if len(line)>0]
    
    
    #combine the name and the begining of their script
    newscript=[]
    k=0
    while k in range(0, len(script)):
        current_word=script[k]
        
        if (current_word[len(current_word)-1]==':') or (k<len(script)-1 and script[k+1][0]==':'):
            newscript.append(current_word+' '+script[k+1])
            k=k+1
        else:
            newscript.append(current_word)
        k=k+1


    
    #combine the [ ]
    script=newscript
    newscript=[]
    k=0
    while k in range(0, len(script)):
        current_word=script[k]
        brack=current_word.find('[')
        if brack>-1 and current_word[brack+1:len(current_word)].find(']')<0:
            hasnt_been_found=True
            s=1
            temp_word=current_word
           
            while hasnt_been_found and k+s<len(script):
                temp_word=temp_word+' '+script[k+s]
                if script[k+s].find(']')>-1:
                    hasnt_been_found=False
                else:
                    s=s+1
                
            newscript.append(temp_word)
            k=k+s
        else:
            newscript.append(current_word)
        k=k+1

    
    #get rid of flashback comment if it is followed by a scene
    script=newscript
    newscript=[]
    k=0
    while k in range(0, len(script)):
        if k<len(script)-1 and script[k].lower().replace(' ','').find('[flash')==0 and script[k+1].lower().replace(' ','').find('[scene')==0: pass
        else: newscript.append(script[k])
        k=k+1

    
    #making sure scene has its own line    
    script=newscript
    newscript=[]
    k=0
    while k in range(0, len(script)):
        current_word=script[k].lower()
        if current_word.replace(' ','').find('[scene')>-1 and current_word.replace(' ','').find('[scene')!=0:
            positon=current_word.find('[scene')
            if positon <0:
                positon=current_word.find('[ scene')
            if positon<0:
                positon=current_word.find('scene')
            newscript.append(current_word[0:positon])
            newscript.append(current_word[positon:len(current_word)])
        else:
            newscript.append(current_word)
        k=k+1
            

    
    #fix more with the :
    script=[line for line in newscript if allowed_after_initial_clean(line)]
    newscript=[]
    k=0
    while k in range(0, len(script)):
        current_word=script[k].lower()
        while k<len(script)-1 and current_word.find(':')>-1 and script[k+1].lower().replace(' ','').find('[scene')<0 and (script[k+1].find(':')<0 or script[k+1].lower().find('sequence')>0) and script[k+1].lower()!='end' and script[k+1].lower()!='the end' and script[k+1].replace(' ','').lower()!='endingcredits':
            current_word=current_word+' '+script[k+1]
            k=k+1
        newscript.append(current_word)
        k=k+1    

        
     
    
    
    templist[pos]=[episode_info, newscript]
    
gc.collect()


0

In [21]:
# we now have the new allwords
allwords=templist

In [22]:
#taking into account sometimes its scene, scene- or scene.
def find_split(n1, n2, n3,n4):
    if -1<n1<n2 and -1<n1<n3:
        return n1
    if -1<n2<n1 and -1<n2<n3:
        return n2
    if -1<n3<n1 and -1<n3<n2:
        return n3
    if -1<n1<n2:
        return n1
    if -1<n1<n3:
        return n1
    if -1<n2<n3:
        return n2
    if -1<n3<n2:
        return n3
    if -1<n1:
        return n1
    if -1<n2:
        return n2
    if -1<n3:
        return n3

    
    return n4

In [23]:
#season 2 epsiode 7,8,10 abbreviated important names
#also sometimes everyone or everybody was used in place of all
def full_clean(text):
    text=text.replace('mnca:','monica:')
    text=text.replace('phoe:','phoebe:')
    text=text.replace('rach:','rachel:')
    text=text.replace('chan:','chandler:')
    text=text.replace('estl:','estelle:')
    
    if text.find('everyone:')==0:
        text='all: '+text[9:len(text)].strip()
    if text.find('everybody:')==0:
        text='all: '+text[10:len(text)].strip()
    if text.find('everybody ')==0:
        text='all '+text[10:len(text)].strip()
    return text

In [24]:
unknown_junk=[]

#if a specific script isn't working, change issue to its index so you get print statemenet along the way
issue=-1

#loop through the script we imported and start making some datasets
for i in range(0, len(allwords)):

  
    season=allwords[i][0][0]
    episode=allwords[i][0][1]
    
    #just so we know our progress
    print([season, episode])
    
    scene_count=0
    location=''
    extra=''
    order=0
    
    #start the loop at 0 and go until we are done
    j=0
    while j in range(0, len(allwords[i][1])):
        gc.collect()
        text=allwords[i][1][j].lower().strip()
        clean_text=full_clean(text)
        
        if issue==episode: print(clean_text)
        
        #did we reach the end of the script
        if clean_text=='ending credits' or clean_text=='end' or clean_text=='the end' or j==len(allwords[i][1])-1:
            scenes.append([season, episode,scene_count, location, extra])
            
            
        #we check if it is a knew scene
        elif clean_text.replace(' ','').find('[scene')==0:
            if scene_count!=0:
                scenes.append([season, episode, scene_count, location, extra])
           
            #start figuring out what it is telling you about the scene
            mini=clean_text.replace('[scene,','').replace('[scene;','').replace('[scene','').replace(']','').replace('[','').replace(':','').strip()
            
            if mini.find(',')>-1 and (mini[0:mini.find(',')]=='ross' or mini[0:mini.find(',')]=='monica' or  mini[0:mini.find(',')]=='chandler'):
                location=" ".join(mini.split())
                extra=''
            else:    
                #does it go scene and then - or . or ,
                split=int(find_split(mini.find(','), mini.find('.'), mini.find('-'),len(mini)))
            
           
            
                location=" ".join(mini[0:split].split())
            
                if split<len(mini)-1:
                    extra=" ".join(mini[split+1:len(mini)].split())
            
            scene_count=scene_count+1
            order=0
            
        #we check if it is a knew scene via a flashback to scene
        elif clean_text.replace(' ','').find('[flashback')==0:
            if scene_count!=0:
                scenes.append([season, episode, scene_count, location, extra])
           
            #start figuring out what it is telling you about the scene
            mini=clean_text.replace('[flashback','').replace(']','').replace('[','').replace(':','').replace('scene','').strip()
            
            #does it go scene and then - or . or ,
            split=int(find_split(mini.find(','), mini.find('.'), mini.find('-'),len(mini)))
            
           
            
            location=" ".join(mini[0:split].split())
            
            if split<len(mini)-1:
                extra=" ".join(mini[split+1:len(mini)].split())
            
            scene_count=scene_count+1
            order=0
            
            
        #its a monica line    
        elif clean_text.replace(' ','').find('monica:')>-1:
            edited_text=clean_text[7:len(clean_text)].replace(':',' ').strip()
            split=''
            
            #find all comments and move them
            while edited_text.find('[')>-1:
                temp_string=edited_text[edited_text.find('['):edited_text.find(']')+1]
                split=split+' '+temp_string
                edited_text=edited_text[0:edited_text.find('[')]+' '+edited_text[edited_text.find(']')+1:len(edited_text)]
            
            Monica.append([season,episode, scene_count, order, " ".join(edited_text.split()),  " ".join(split.split())])
            order=order+1
            
            
        #its a ross line    
        elif clean_text.replace(' ','').find('ross:')>-1:
            edited_text=clean_text[5:len(clean_text)].replace(':',' ').strip()
            split=''
            
            #find all comments and move them
            while edited_text.find('[')>-1:
                temp_string=edited_text[edited_text.find('['):edited_text.find(']')+1]
                split=split+' '+temp_string
                edited_text=edited_text[0:edited_text.find('[')]+' '+edited_text[edited_text.find(']')+1:len(edited_text)]
            
            Ross.append([season,episode, scene_count, order," ".join(edited_text.split()),  " ".join(split.split())])
            order=order+1
            

            
            
        #its a phoebe line    
        elif clean_text.replace(' ','').find('phoebe:')>-1:
            edited_text=clean_text[7:len(clean_text)].replace(':',' ').strip()
            split=''
            
            #find all comments and move them
            while edited_text.find('[')>-1:
                temp_string=edited_text[edited_text.find('['):edited_text.find(']')+1]
                split=split+' '+temp_string
                edited_text=edited_text[0:edited_text.find('[')]+' '+edited_text[edited_text.find(']')+1:len(edited_text)]
            
            Phoebe.append([season,episode, scene_count, order, " ".join(edited_text.split()),  " ".join(split.split())])
            order=order+1
            
            
        #its a joey line    
        elif clean_text.replace(' ','').find('joey:')>-1:
            edited_text=clean_text[5:len(clean_text)].replace(':',' ').strip()
            split=''
            
            #find all comments and move them
            while edited_text.find('[')>-1:
                temp_string=edited_text[edited_text.find('['):edited_text.find(']')+1]
                split=split+' '+temp_string
                edited_text=edited_text[0:edited_text.find('[')]+' '+edited_text[edited_text.find(']')+1:len(edited_text)]
            
            Joey.append([season,episode, scene_count, order, " ".join(edited_text.split()),  " ".join(split.split())])
            order=order+1
            
            
        #its a rachel line    
        elif clean_text.replace(' ','').find('rachel:')>-1:
            edited_text=clean_text[7:len(clean_text)].replace(':',' ').strip()
            split=''
            
            #find all comments and move them
            while edited_text.find('[')>-1:
                temp_string=edited_text[edited_text.find('['):edited_text.find(']')+1]
                split=split+' '+temp_string
                edited_text=edited_text[0:edited_text.find('[')]+' '+edited_text[edited_text.find(']')+1:len(edited_text)]
            
            Rachel.append([season,episode, scene_count, order, " ".join(edited_text.split()),  " ".join(split.split())])
            order=order+1
            
        #its a chandler line    
        elif clean_text.replace(' ','').find('chandler:')>-1:
            edited_text=clean_text[9:len(clean_text)].replace(':',' ').strip()
            split=''
            
            #find all comments and move them
            while edited_text.find('[')>-1:
                temp_string=edited_text[edited_text.find('['):edited_text.find(']')+1]
                split=split+' '+temp_string
                edited_text=edited_text[0:edited_text.find('[')]+' '+edited_text[edited_text.find(']')+1:len(edited_text)]
                
            
            Chandler.append([season,episode, scene_count, order, " ".join(edited_text.split()),  " ".join(split.split())])
            order=order+1
            
        #its a all line    
        elif clean_text.replace(' ','').find('all:')>-1:
            edited_text=clean_text[4:len(clean_text)].replace(':',' ').strip()
            split=''
            
            #find all comments and move them
            while edited_text.find('[')>-1:
                temp_string=edited_text[edited_text.find('['):edited_text.find(']')+1]
                split=split+' '+temp_string
                edited_text=edited_text[0:edited_text.find('[')]+' '+edited_text[edited_text.find(']')+1:len(edited_text)]
            
            All.append([season,episode, scene_count, order, " ".join(edited_text.split()),  " ".join(split.split())])
            order=order+1
        
            
        #minor character
        elif clean_text.find(':')>-1 and clean_text.replace(' ','').find('matthewperry')<0 and clean_text.replace(' ','').find('lisakudrow')<0:
            name= clean_text[0:clean_text.find(':')].replace('mr.','mr').replace('mrs.','mrs')
            edited_text=clean_text[len(name)+1:len(clean_text)]
            split=''
            
            #find all comments and move them
            while edited_text.find('[')>-1:
                temp_string=edited_text[edited_text.find('['):edited_text.find(']')+1]
                split=split+' '+temp_string
                edited_text=edited_text[0:edited_text.find('[')]+' '+edited_text[edited_text.find(']')+1:len(edited_text)]
                
            minors.append([season, episode, scene_count,  order, name," ".join(edited_text.split()),  " ".join(split.split())])
            order=order+1
        
        
            #do we have the name already in the list of all_char names?
            isit=[n for n in all_char if n[0]==name]
            if len(isit)==0:
                all_char.append([name, False])
                
        else:
            #not my problem?
            unknown_junk.append(clean_text)
            
        j=j+1

[1, 1]
[1, 2]
[1, 3]
[1, 4]
[1, 5]
[1, 6]
[1, 7]
[1, 8]
[1, 9]
[1, 10]
[1, 11]
[1, 12]
[1, 13]
[1, 14]
[1, 15]
[1, 16]
[1, 17]
[1, 18]
[1, 19]
[1, 20]
[1, 21]
[1, 22]
[1, 23]
[1, 24]
[2, 1]
[2, 2]
[2, 3]
[2, 4]
[2, 5]
[2, 6]
[2, 7]
[2, 8]
[2, 9]
[2, 10]
[2, 11]
[2, 12]
[2, 14]
[2, 15]
[2, 16]
[2, 17]
[2, 18]
[2, 19]
[2, 20]
[2, 21]
[2, 22]
[2, 23]
[2, 24]
[3, 1]
[3, 2]
[3, 3]
[3, 4]
[3, 5]
[3, 6]
[3, 7]
[3, 8]
[3, 9]
[3, 10]
[3, 11]
[3, 12]
[3, 13]
[3, 14]
[3, 15]
[3, 16]
[3, 17]
[3, 18]
[3, 19]
[3, 20]
[3, 21]
[3, 22]
[3, 23]
[3, 24]
[3, 25]
[4, 1]
[4, 2]
[4, 3]
[4, 4]
[4, 5]
[4, 6]
[4, 7]
[4, 8]
[4, 9]
[4, 10]
[4, 11]
[4, 12]
[4, 13]
[4, 14]
[4, 15]
[4, 16]
[4, 17]
[4, 18]
[4, 19]
[4, 20]
[4, 21]
[4, 22]
[4, 23]
[5, 1]
[5, 2]
[5, 3]
[5, 4]
[5, 5]
[5, 6]
[5, 7]
[5, 8]
[5, 9]
[5, 10]
[5, 11]
[5, 12]
[5, 13]
[5, 14]
[5, 15]
[5, 16]
[5, 17]
[5, 18]
[5, 19]
[5, 20]
[5, 21]
[5, 22]
[5, 23]
[6, 1]
[6, 2]
[6, 3]
[6, 4]
[6, 5]
[6, 6]
[6, 7]
[6, 8]
[6, 9]
[6, 10]
[6, 11]
[6, 12]
[6, 13]
[6, 14

In [25]:
pd.DataFrame({'season': [m[0] for m in Monica],'episode': [m[1] for m in Monica],'scene': [m[2] for m in Monica],'quote_index': [m[3] for m in Monica],'line': [m[4] for m in Monica],'extra': [m[5] for m in Monica]}).to_csv('../data/Characters/Monica.csv', index=False)
pd.DataFrame({'season': [m[0] for m in Ross],'episode': [m[1] for m in Ross],'scene': [m[2] for m in Ross],'quote_index': [m[3] for m in Ross],'line': [m[4] for m in Ross],'extra': [m[5] for m in Ross]}).to_csv('../data/Characters/Ross.csv', index=False)
pd.DataFrame({'season': [m[0] for m in Phoebe],'episode': [m[1] for m in Phoebe],'scene': [m[2] for m in Phoebe],'quote_index': [m[3] for m in Phoebe],'line': [m[4] for m in Phoebe],'extra': [m[5] for m in Phoebe]}).to_csv('../data/Characters/Phoebe.csv', index=False)
pd.DataFrame({'season': [m[0] for m in Joey],'episode': [m[1] for m in Joey],'scene': [m[2] for m in Joey],'quote_index': [m[3] for m in Joey],'line': [m[4] for m in Joey],'extra': [m[5] for m in Joey]}).to_csv('../data/Characters/Joey.csv', index=False)
pd.DataFrame({'season': [m[0] for m in Rachel],'episode': [m[1] for m in Rachel],'scene': [m[2] for m in Rachel],'quote_index': [m[3] for m in Rachel],'line': [m[4] for m in Rachel],'extra': [m[5] for m in Rachel]}).to_csv('../data/Characters/Rachel.csv', index=False)
pd.DataFrame({'season': [m[0] for m in Chandler],'episode': [m[1] for m in Chandler],'scene': [m[2] for m in Chandler],'quote_index': [m[3] for m in Chandler],'line': [m[4] for m in Chandler],'extra': [m[5] for m in Chandler]}).to_csv('../data/Characters/Chandler.csv', index=False)
pd.DataFrame({'season': [m[0] for m in All],'episode': [m[1] for m in All],'scene': [m[2] for m in All],'quote_index': [m[3] for m in All],'line': [m[4] for m in All],'extra': [m[5] for m in All]}).to_csv('../data/Characters/All.csv', index=False)


#For the Scene DataBase
pd.DataFrame({'season': [m[0] for m in scenes],'episode': [m[1] for m in scenes],'scene': [m[2] for m in scenes],'location': [m[3] for m in scenes],"extra_scene_info": [m[4] for m in scenes]}).to_csv('../data/Characters/scenes.csv', index=False)

#For the Minor Chatacters DataBasepd.DataFrame({'name':[m[0] for m in all_char],'main':[m[1] for m in all_char]})
pd.DataFrame({'season': [m[0] for m in minors],'episode': [m[1] for m in minors],'scene': [m[2] for m in minors],'quote_index': [m[3] for m in minors],'character': [m[4] for m in minors],"line": [m[5] for m in minors],'extra': [m[6] for m in minors]}).to_csv('../data/Characters/minors.csv', index=False)

#For the All Chatacters DataBase
pd.DataFrame({'name':[m[0] for m in all_char],'main':[m[1] for m in all_char]}).to_csv('../data/Characters/all_char.csv', index=False)
